 # Data Loading, Storage, & File Formats

 `pandas` features a number of functions for reading tabular data as a DataFrame object.

In [1]:
import pandas as pd
import numpy as np
import sys
import csv
import json
from lxml import objectify
import requests
import re
import sqlite3
import sqlalchemy as sqla

In [2]:
ex1 = "examples/ex1.csv"

In [3]:
! cat $ex1

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
test_df1 = pd.read_csv(ex1)
test_df1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 Use `header` parameter to set any row as the column labels or `names` to set custom column labels:

In [5]:
# Creates column index [0:)
df_col_names = ["a", "b", "c", "d", "message"]
test_df2 = pd.read_csv("examples/ex2.csv", header=None, names=df_col_names)
test_df2

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 Let's say we want the "message" column as the index

In [6]:
pd.read_csv("examples/ex2.csv", names=df_col_names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


 Hierarchical Index:

In [7]:
pd.read_csv("examples/csv_mindex.csv", index_col=["key1", "key2"])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

 Some times a table might not have a fixed or use some other delimiter, we can still parse that data:

In [8]:
pd.read_csv("examples/ex3.txt", sep="\s+")

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


 The `sep` argument can take a regular expression as well as a string.

 Because there is one fewer column name (first row) than the columns (all other rows), pandas infers the first column as the index.

In [9]:
pd.read_csv("examples/ex4.csv", skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 Handling missing data while reading a file is important. By default, `pandas` parses blank space, or *sentinels* such as NA & NULL.

In [10]:
! cat "examples/ex5.csv"
pd.read_csv("examples/ex5.csv")

something,a,b,c,d,message
one,,null,3,4,NA
two,5,6,,8,world
three,NULL,10,11,12,foo

,something,a,b,c,d,message
0,one,NaN,NaN,3.0,4,NaN
1,two,5.0,6.0,NaN,8,world
2,three,NaN,10.0,11.0,12,foo


In [11]:
pd.read_csv("examples/ex5.csv", na_values=["foo"])

,something,a,b,c,d,message
0,one,NaN,NaN,3.0,4,NaN
1,two,5.0,6.0,NaN,8,world
2,three,NaN,10.0,11.0,12,NaN


 We can add more sentinels by using the `na_values` argument.

In [12]:
pd.read_csv("examples/ex5.csv", keep_default_na=False)

,something,a,b,c,d,message
0,one,,null,3,4,NA
1,two,5,6,,8,world
2,three,NULL,10,11,12,foo


In [13]:
df5 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["NA", "null", "", "NULL"])
df5

,something,a,b,c,d,message
0,one,NaN,NaN,3.0,4,NaN
1,two,5.0,6.0,NaN,8,world
2,three,NaN,10.0,11.0,12,foo


In [14]:
df5.isna()

,something,a,b,c,d,message
0,False,True,True,False,False,True
1,False,False,False,True,False,False
2,False,True,False,False,False,False


 We can even define sentinels specific to columns:

In [15]:
sentinels = {
  "something": ["two"], 
  "message": ["foo"]
}
pd.read_csv("examples/ex5.csv", na_values=sentinels)

,something,a,b,c,d,message
0,one,NaN,NaN,3.0,4,NaN
1,NaN,5.0,6.0,NaN,8,world
2,three,NaN,10.0,11.0,12,NaN


 ### Reading text (CSV) files in chunks

In [16]:
# Read small piece of a large file:
pd.options.display.max_rows = 10

In [17]:
pd.read_csv("examples/ex6.csv")

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


 Read in chunks of X rows:

In [18]:
df6_chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
key_fq = pd.Series([], dtype="int64") # used later
type(df6_chunker)

pandas.io.parsers.readers.TextFileReader

In [19]:
for chunk in df6_chunker:
  print(chunk)
  break

          one       two     three      four key
0    0.467976 -0.038649 -0.295344 -1.824726   L
1   -0.358893  1.404453  0.704965 -0.200638   B
2   -0.501840  0.659254 -0.421691 -0.057688   G
3    0.204886  1.074134  1.388361 -0.982404   R
4    0.354628 -0.133116  0.283763 -0.837063   Q
..        ...       ...       ...       ...  ..
995  2.311896 -0.417070 -1.409599 -0.515821   M
996 -0.479893 -0.650419  0.745152 -0.646038   H
997  0.523331  0.787112  0.486066  1.093156   D
998 -0.362559  0.598894 -1.843201  0.887292   W
999 -0.096376 -1.012999 -0.657431 -0.573315   K

[1000 rows x 5 columns]


 Or open it as a handler, then iterate over the chunks:

In [20]:
with pd.read_csv("examples/ex6.csv", chunksize=1000) as reader:
  # Read chuncksize rows at every iteration
  for chunk in reader:
    # # Check df
    # print(chunk.head())
    # break
    # Adding series (+), fill_value: key_fq was a blank Series.
    key_fq = key_fq.add(chunk["key"].value_counts(), fill_value=0)

key_fq

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
     ...  
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
Length: 36, dtype: float64

In [22]:
with pd.read_csv("examples/ex6.csv", chunksize=1000) as reader:
  # Can iterate like: for chunk in reader.get_chunk()
  # It's a generater function
  print(reader.get_chunk())

          one       two     three      four key
0    0.467976 -0.038649 -0.295344 -1.824726   L
1   -0.358893  1.404453  0.704965 -0.200638   B
2   -0.501840  0.659254 -0.421691 -0.057688   G
3    0.204886  1.074134  1.388361 -0.982404   R
4    0.354628 -0.133116  0.283763 -0.837063   Q
..        ...       ...       ...       ...  ..
995  2.311896 -0.417070 -1.409599 -0.515821   M
996 -0.479893 -0.650419  0.745152 -0.646038   H
997  0.523331  0.787112  0.486066  1.093156   D
998 -0.362559  0.598894 -1.843201  0.887292   W
999 -0.096376 -1.012999 -0.657431 -0.573315   K

[1000 rows x 5 columns]


 ### Writing out text files

In [23]:
df5

,something,a,b,c,d,message
0,one,NaN,NaN,3.0,4,NaN
1,two,5.0,6.0,NaN,8,world
2,three,NaN,10.0,11.0,12,foo


In [24]:
df5.to_csv("examples/out.csv")
! cat examples/out.csv

,something,a,b,c,d,message
0,one,,,3.0,4,
1,two,5.0,6.0,,8,world
2,three,,10.0,11.0,12,foo


In [25]:
# Other delimiters & represent nan values as "NULL":
df5.to_csv(sys.stdout, sep="|", na_rep="NULL")

|something|a|b|c|d|message
0|one|NULL|NULL|3.0|4|NULL
1|two|5.0|6.0|NULL|8|world
2|three|NULL|10.0|11.0|12|foo


In [26]:
# Can disable index & headers
df5.to_csv(sys.stdout, index=False, header=False) # header = bool or list

one,,,3.0,4,
two,5.0,6.0,,8,world
three,,10.0,11.0,12,foo


In [27]:
# Only a subset of cols
df5.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
,,3.0
5.0,6.0,
,10.0,11.0


In [28]:
! cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [29]:
pd.read_csv("examples/ex7.csv")

,a,b,c
0,1,2,3
1,1,2,3


In [30]:
with open("examples/ex7.csv") as f:
  for line in csv.reader(f):
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [31]:
with open("examples/ex7.csv") as f:
  lines = list(csv.reader(f))
  header, values = lines[0], lines[1:]
  data_dict = {h: v for h, v in zip(header, zip(*values))}
  print(data_dict)

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


In [32]:
class custom_dialect(csv.Dialect):
  # refer the docs
  delimiter = ";"
  lineterminator = "\n"
  quotechar = '"'
  quoting = csv.QUOTE_MINIMAL

 Now, we can write a csv file with a custom format, dialect:

In [33]:
with open("examples/custom_dialect_file.csv", "w") as file:
  writer = csv.writer(file, dialect=custom_dialect)
  writer.writerow(("one", "two", "three"))
  writer.writerow(tuple("123"))
  writer.writerow(tuple("456"))

 ### JSON Files

In [34]:
obj_js = """
{
  "name": "Vikram Negi",
  "cities_lived": ["Navi Mumbai", "Mumbai", "Dehradun"],
  "pet": null,
  "siblings": [
    {"name": "Yogesh", "age": 16, "hobbies": ["gaming", "gym", "anime", "football"]},
    {"name": "Varun", "age": 5, "hobbies": ["reading", "gaming", "cartoon"]}
  ]
}
"""
test_js = json.loads(obj_js)
test_js

{'name': 'Vikram Negi',
 'cities_lived': ['Navi Mumbai', 'Mumbai', 'Dehradun'],
 'pet': None,
 'siblings': [{'name': 'Yogesh',
   'age': 16,
   'hobbies': ['gaming', 'gym', 'anime', 'football']},
  {'name': 'Varun', 'age': 5, 'hobbies': ['reading', 'gaming', 'cartoon']}]}

In [35]:
sibs = pd.DataFrame(test_js["siblings"], columns=["name", "age"])
sibs

,name,age
0,Yogesh,16
1,Varun,5


In [36]:
! cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [37]:
js = pd.read_json("examples/example.json")
js

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [38]:
js.to_json(sys.stdout)
# Or, for array output:
js.to_json(sys.stdout, orient="records")

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

 ### HTML & XML: Web Scrapping

In [39]:
# Need to install lxml module
tables = pd.read_html("examples/fdic_failed_bank_list.html")
failed_banks = tables[0]

In [40]:
failed_banks.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


 No. of bank failures by year:

In [41]:
closing_timestamps = pd.to_datetime(failed_banks["Closing Date"])
closing_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [42]:
with open("datasets/mta_perf/Performance_MNR.xml") as f:
  parsed = objectify.parse(f)

In [43]:
root = parsed.getroot()
root

<Element PERFORMANCE at 0x7fbeec9bfb40>

In [44]:
xml_data = []
skip_fields = ("INDICATOR_SEQ", "PARENT_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES")

for el in root.INDICATOR:
  el_data = {}
  for child in el.getchildren():
    if child.tag not in skip_fields:
      # using .pyval instead of .text, because it type converts the values
      el_data[child.tag] = child.pyval

  if len(el_data) != 0:
    xml_data.append(el_data)

len(xml_data)

648

In [45]:
xml_df = pd.DataFrame(xml_data)
xml_df.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


 The above data extraction can be done in a single line using `.read_xml` method in pandas:

In [46]:
xml_df2 = pd.read_xml("datasets/mta_perf/Performance_MNR.xml")

include_fields = []
for field in xml_df2.columns:
  if field not in skip_fields:
    include_fields.append(field)

xml_df2.loc[:, include_fields].head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.00,96.90,95.00,96.90
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.00,96.00,95.00,95.00
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.00,96.30,95.00,96.90
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.00,96.80,95.00,98.30
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.00,96.60,95.00,95.80


 ### Binary Data Format

In [47]:
# Method 1: Pickle format
df_1 = pd.read_csv("examples/ex1.csv")
df_1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [48]:
df_1.to_pickle("examples/ex_pickle")

In [49]:
pd.read_pickle("examples/ex_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 `pickle` is only recommended as a short-term storage format, because it may not be supported in the future version of python.

 ### Excel files

In [50]:
# Used when multiple sheets in an xlsx file
xlsx = pd.ExcelFile("examples/ex1.xlsx")
excel_sheet_name = xlsx.sheet_names[0]
xlsx.parse(excel_sheet_name, index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [51]:
# OR simply use read_excel
xl_frame = pd.read_excel("examples/ex1.xlsx", sheet_name=excel_sheet_name, index_col=0)

 Similarly there are two ways to write data to an excel file:

In [52]:
xlsx_writer = pd.ExcelWriter("examples/ex2.xlsx")
xl_frame.to_excel(xlsx_writer, "Sheet1")
xlsx_writer.save()

In [53]:
pd.read_excel("examples/ex2.xlsx", sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


 ### Hierarchial Data Format

 First install `tables` package

In [54]:
test_df = pd.DataFrame(np.random.standard_normal((100,5))*5, columns=list("abcde"))
test_df.head()

,a,b,c,d,e
0,5.606668,5.449402,-0.728420,3.286665,-4.803210
1,-3.372551,-1.206116,5.507383,-2.990513,-0.031590
2,-2.204594,2.803767,-4.137105,0.371271,4.814977
3,-1.220577,-11.397113,-1.823349,1.215034,-1.821504
4,-1.242612,-2.628863,-3.530020,-3.065260,7.033785


In [55]:
# Store class
store = pd.HDFStore("examples/test.h5")
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/test.h5

In [56]:
store["test_1"] = test_df # store DataFrame
store["col_a"] = test_df["a"] # store Series

In [57]:
store["test_1"].head()

,a,b,c,d,e
0,5.606668,5.449402,-0.728420,3.286665,-4.803210
1,-3.372551,-1.206116,5.507383,-2.990513,-0.031590
2,-2.204594,2.803767,-4.137105,0.371271,4.814977
3,-1.220577,-11.397113,-1.823349,1.215034,-1.821504
4,-1.242612,-2.628863,-3.530020,-3.065260,7.033785


 `HDFStore` supports two storage schemas (format), "`fixed`" & "`table`"

 * Default is `fixed`
 * `table` is slower, but supports query operations using special syntax

In [58]:
# put is the same as assigning store["key_name"] = df, but allows for defining
# specific format
store.put("test_2", test_df, format="table")

In [59]:
# Query it kinda like a database
store.select("test_2", where=["index >= 10 and index < 15"])

,a,b,c,d,e
10,1.506609,-4.751056,-9.368857,-5.356335,-3.575709
11,-5.942064,2.519561,-0.150001,-5.999577,-2.481324
12,6.554236,-1.262817,-1.697952,3.749887,4.184971
13,5.071260,-3.113428,1.511293,-1.518928,-7.893768
14,-1.236928,-0.061157,0.252540,-8.637764,4.939822


 Here is a simple way to do read & write HDF:

In [61]:
# Write operation
test_df.to_hdf("examples/test2.h5", "test", format="table")

In [62]:
# Read operation
pd.read_hdf("examples/test2.h5", "test", where=["index < 5"])

,a,b,c,d,e
0,5.606668,5.449402,-0.728420,3.286665,-4.803210
1,-3.372551,-1.206116,5.507383,-2.990513,-0.031590
2,-2.204594,2.803767,-4.137105,0.371271,4.814977
3,-1.220577,-11.397113,-1.823349,1.215034,-1.821504
4,-1.242612,-2.628863,-3.530020,-3.065260,7.033785


 **Note**: `HDF5` isn't a database. It's best suited for write-once, read-many datasets.

 ### Interacting with Web APIs

In [63]:
reddit_tiktok_url = "https://www.reddit.com/r/TikTokCringe/top.json?limit=10"
headers = {"user-agent": "Linux Machine (Vikram Singh Negi)"}

res = requests.get(reddit_tiktok_url, headers=headers)
try:
  res.raise_for_status()
  data = res.json()["data"]
except Exception as err:
  print(f"HTTP Error: {err}")

In [64]:
# pd.DataFrame([child["data"] for child in data["children"]])
[child["data"] for child in data["children"]][0]

{'approved_at_utc': None,
 'subreddit': 'TikTokCringe',
 'selftext': '',
 'author_fullname': 't2_6wdexyp2',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'New dress',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/TikTokCringe',
 'hidden': False,
 'pwls': 7,
 'link_flair_css_class': '',
 'downs': 0,
 'thumbnail_height': 140,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_vvvjgv',
 'quarantine': False,
 'link_flair_text_color': 'light',
 'upvote_ratio': 0.92,
 'author_flair_background_color': None,
 'ups': 21103,
 'total_awards_received': 1,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': {'reddit_video': {'bitrate_kbps': 2400,
   'fallback_url': 'https://v.redd.it/a1v6p2lvrra91/DASH_720.mp4?source=fallback',
   'height': 720,
   'width': 405,
   'scrubber_media_url': 'https://v.redd.it/a1v6p2lvrra91/DASH_96.mp4',
   'da

In [65]:
dt = []
valid_fields = ["subreddit", "title", "thumbnail", "url_overridden_by_dest", "subreddit_id", "author", "url", "media", "is_video"]

for child in data["children"]:
  if child["data"]["is_video"]:
    child_data = {}
    for key, val in child["data"].items():
      if key in valid_fields:
        if key == "media":
          fallback_url = val["reddit_video"]["fallback_url"]
          child_data["video_url"] = fallback_url
          child_data["audio_url"] = re.sub(r"[\w+\/]DASH_(\d+)", "/DASH_audio", fallback_url)
        else:
          child_data[key] = val
    dt.append(child_data)

pd.DataFrame(dt)

,subreddit,title,thumbnail,url_overridden_by_dest,subreddit_id,author,url,video_url,audio_url,is_video
0,TikTokCringe,New dress,https://b.thumbs.redditmedia.com/nb4y_Vh1bXGUY...,https://v.redd.it/a1v6p2lvrra91,t5_mvcq5,lilmcfuggin,https://v.redd.it/a1v6p2lvrra91,https://v.redd.it/a1v6p2lvrra91/DASH_720.mp4?s...,https://v.redd.it/a1v6p2lvrra91/DASH_audio.mp4...,True
1,TikTokCringe,Why does this remind me of the purge.,https://a.thumbs.redditmedia.com/jwKt35ny--rp4...,https://v.redd.it/syfpj2y0rta91,t5_mvcq5,_ElReaper,https://v.redd.it/syfpj2y0rta91,https://v.redd.it/syfpj2y0rta91/DASH_360.mp4?s...,https://v.redd.it/syfpj2y0rta91/DASH_audio.mp4...,True
2,TikTokCringe,Sisters pregnant at the same time,https://a.thumbs.redditmedia.com/lmxVX7Vzg6KH4...,https://v.redd.it/vw7jf6vz4ta91,t5_mvcq5,bigdickjenny,https://v.redd.it/vw7jf6vz4ta91,https://v.redd.it/vw7jf6vz4ta91/DASH_1080.mp4?...,https://v.redd.it/vw7jf6vz4ta91/DASH_audio.mp4...,True
3,TikTokCringe,The two brain cells trying to form together 😂 ...,https://a.thumbs.redditmedia.com/34pwq5a0OTunX...,https://v.redd.it/z52y2vzwrsa91,t5_mvcq5,megami96,https://v.redd.it/z52y2vzwrsa91,https://v.redd.it/z52y2vzwrsa91/DASH_720.mp4?s...,https://v.redd.it/z52y2vzwrsa91/DASH_audio.mp4...,True
4,TikTokCringe,She was NOT pleased!,https://b.thumbs.redditmedia.com/4lGGyPYO679Z7...,https://v.redd.it/4r2wjfd5rua91,t5_mvcq5,WaterEnvironmental80,https://v.redd.it/4r2wjfd5rua91,https://v.redd.it/4r2wjfd5rua91/DASH_720.mp4?s...,https://v.redd.it/4r2wjfd5rua91/DASH_audio.mp4...,True
5,TikTokCringe,Bear encounters a mirror,https://b.thumbs.redditmedia.com/OIxZNt0FqnBTX...,https://v.redd.it/7if2610fjqa91,t5_mvcq5,LeSpatula,https://v.redd.it/7if2610fjqa91,https://v.redd.it/7if2610fjqa91/DASH_480.mp4?s...,https://v.redd.it/7if2610fjqa91/DASH_audio.mp4...,True
6,TikTokCringe,How To Make Buttermilk Biscuits,https://b.thumbs.redditmedia.com/HKV9bwB-FEAO3...,https://v.redd.it/pkal0mnx8sa91,t5_mvcq5,Artistic-Audience182,https://v.redd.it/pkal0mnx8sa91,https://v.redd.it/pkal0mnx8sa91/DASH_1080.mp4?...,https://v.redd.it/pkal0mnx8sa91/DASH_audio.mp4...,True
7,TikTokCringe,Natural hair,https://a.thumbs.redditmedia.com/EgjEc7FcFDj_V...,https://v.redd.it/mm2zcrmesra91,t5_mvcq5,lilmcfuggin,https://v.redd.it/mm2zcrmesra91,https://v.redd.it/mm2zcrmesra91/DASH_720.mp4?s...,https://v.redd.it/mm2zcrmesra91/DASH_audio.mp4...,True
8,TikTokCringe,Attempting to pick on a bassist,https://b.thumbs.redditmedia.com/CjU3gqkdDVBKr...,https://v.redd.it/iwj5g7ikvpa91,t5_mvcq5,EndVry,https://v.redd.it/iwj5g7ikvpa91,https://v.redd.it/iwj5g7ikvpa91/DASH_720.mp4?s...,https://v.redd.it/iwj5g7ikvpa91/DASH_audio.mp4...,True
9,TikTokCringe,Cubert the attention hog,https://b.thumbs.redditmedia.com/VC6Po2zTjJHU6...,https://v.redd.it/jxsptjl9sra91,t5_mvcq5,lilmcfuggin,https://v.redd.it/jxsptjl9sra91,https://v.redd.it/jxsptjl9sra91/DASH_720.mp4?s...,https://v.redd.it/jxsptjl9sra91/DASH_audio.mp4...,True


 ### Interacting with Databases

In [66]:
con = sqlite3.connect("examples/test.db")

In [67]:
create_table_query = """
  CREATE TABLE us_cities (
    city VARCHAR(20),
    state VARCHAR(20),
    population REAL,
    rating INTEGER
  )
"""
try:
  con.execute(create_table_query)
  con.commit()
except Exception as err:
  print("DB Error:", err)

In [68]:
db_data = [
  ("Atlanta", "Georgia", 1.25, 6),
  ("Tallahassee", "Florida", 2.6, 3),
  ("Sacramento", "California", 1.7, 5)
]

con.executemany("INSERT INTO us_cities VALUES (?, ?, ?, ?)", db_data)
con.commit()

 Read from database:

In [69]:
cur = con.execute("SELECT * FROM us_cities")
rows = cur.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [70]:
cur.description

(('city', None, None, None, None, None, None),
 ('state', None, None, None, None, None, None),
 ('population', None, None, None, None, None, None),
 ('rating', None, None, None, None, None, None))

In [71]:
pd.DataFrame(rows, columns=[name[0] for name in cur.description])

,city,state,population,rating
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


 Using SQLAlchemy to make the above process simple:

In [72]:
db = sqla.create_engine("sqlite:///examples/test.db")
pd.read_sql("SELECT * FROM us_cities", db)

,city,state,population,rating
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
